### Environment Preparation

In [ ]:
# code to set the python version to 3.8
# !sudo update-alternatives --config python3
# !python -V
# !sudo apt-get install python3-pip
# !python -m pip install --upgrade pip
# !pip install ipykernel

In [ ]:
!pip install torch torchvision transformers
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached wandb-0.15.1-py3-none-any.whl (2.0 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 19.9 MB/s eta 0:00:00
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
  Using cached setproctitle-1.3.2-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (30 kB)
  Using cached sentry_sdk-1.22.0-py2.py3-none-any.whl (203 kB)
  Using cached pathtools-0.1.2.tar.gz (11 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.3 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=57acefac9d13473e377bae99b4c9a831d03d6d6f336953522e43dd6456ffda50
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56f

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from torch.utils.data import Dataset
import json
import random
from tqdm import tqdm
import torch.nn.functional as F
import torch
import numpy as np
from datetime import datetime
from transformers import AutoTokenizer, AutoModel
import wandb
from torch.utils.data import DataLoader
import torch.optim as optim
import os
import torch.nn as nn

### Evidence Retrieval Dataset

In [ ]:
# function to merge two dictionaries
def Merge(dict1, dict2):
    res = {**dict1, **dict2}
    return res

In [ ]:
# Process the training dataset
class TrainDataset(Dataset):
  def __init__(self, mode, tokenizer, evidence_samples, max_length):
    self.mode = mode
    self.tokenizer = tokenizer
    self.evidence_samples = evidence_samples
    self.max_length = max_length
    
    # use both the train and dev datasets to train the model
    if mode == "train_dev":
      f = open("./drive/MyDrive/NLP_ass3/data/train-claims.json", "r")
      train_dataset = json.load(f)
      f.close()

      f = open("./drive/MyDrive/NLP_ass3/data/dev-claims.json", "r")
      dev_dataset = json.load(f)
      f.close()

      self.dataset = Merge(train_dataset, dev_dataset)
    else:
      # open the train/dev file
      f = open("./drive/MyDrive/NLP_ass3/data/{}-claims.json".format(mode), "r")
      self.dataset = json.load(f)
      f.close()
    self.claim_ids = list(self.dataset.keys())
    
    # open the evidence file 
    f = open("./drive/MyDrive/NLP_ass3/data/evidence.json", "r")
    self.evidences = json.load(f)
    f.close()
    self.evidence_ids = list(self.evidences.keys())

  def __len__(self):
    return len(self.claim_ids)

  def __getitem__(self, index):
    claim_id = self.claim_ids[index]
    data = self.dataset[claim_id]
    processed_query = data["claim_text"].lower()
    evidences = []
    for evidence_id in data["evidences"]:
      evidences.append(evidence_id)
    return [processed_query, evidences]

  def collate_fn(self, batch):
    queries = []
    evidences = []
    answer_lens = []
    for query, evidence in batch:
      queries.append(query)
      evidences.extend(evidence)
      answer_lens.append(len(evidence))
    
    # set some negative example for training
    evidence_num = len(evidences)
    # if the number of evidence is larger than example setting, trancate the evidence list
    if evidence_num > self.evidence_samples:
      evidences = evidences[:self.evidence_samples]

    # if the number of evidence is less than example setting,
    # randomly choose evidence from the dataset while proving there is no duplication
    evidences_text = [self.evidences[evidence_id].lower() for evidence_id in evidences]
    while evidence_num < self.evidence_samples:
      evidence_id = random.choice(self.evidence_ids)
      while evidence_id in evidences:
        evidence_id = random.choice(self.evidence_ids)
      evidences.append(evidence_id)
      evidences_text.append(self.evidences[evidence_id].lower())
      evidence_num += 1

    query_text_token = self.tokenizer(
        queries,
        max_length=self.max_length,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )

    evidences_text_token = self.tokenizer(
        evidences_text,
        max_length=self.max_length,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )

    encoding_dict = {"query_input_ids" : query_text_token["input_ids"],
            "evidence_input_ids" : evidences_text_token["input_ids"],
            "query_attention_mask" : query_text_token["attention_mask"],
            "evidence_attention_mask" : evidences_text_token["attention_mask"],
            "answer_lens" : answer_lens}
    return encoding_dict


In [ ]:
# Process the validate dataset
class ValidateDataset(Dataset):
  def __init__(self, mode, tokenizer, max_length):
    self.tokenizer = tokenizer
    self.mode = mode
    self.max_length = max_length

    # open the dev/test file
    if mode != "test":
      f = open("./drive/MyDrive/NLP_ass3/data/{}-claims.json".format(mode), "r")
    else:
      f = open("./drive/MyDrive/NLP_ass3/data/test-claims-unlabelled.json", "r")

    self.dataset = json.load(f)
    f.close()

    # read the claim ids to a list
    self.claim_ids = list(self.dataset.keys())

  def __len__(self):
    return len(self.claim_ids)
  
  def __getitem__(self, index):
    claim_id = self.claim_ids[index]
    data = self.dataset[claim_id]
    processed_query = data["claim_text"].lower()
    return [processed_query, data, claim_id]

  def collate_fn(self, batch):
    queries = []
    datas = []
    claim_ids = []
    evidences = []
    # print("read the dev file 1111")

    for query, data, claim_id in batch:
      queries.append(query)
      datas.append(data)
      claim_ids.append(claim_id)
      if self.mode != "test":
        evidences.append(data["evidences"])

    # print("read the dev file")

    query_text_token = self.tokenizer(
      queries,
      max_length=self.max_length,
      padding=True,
      truncation=True,
      return_tensors="pt"
    )

    encoding_dict = {"query_input_ids" : query_text_token["input_ids"],
              "query_attention_mask" : query_text_token["attention_mask"],
              "datas" : datas,
              "claim_ids" : claim_ids}
    # if the file is dev, encode the evidences into the encodings
    if self.mode != "test":
      encoding_dict["evidences"] = evidences
    return encoding_dict


In [ ]:
# Process the evidence dataset
class EvidenceDataset(Dataset):
  def __init__(self, tokenizer, max_length):
    self.tokenizer = tokenizer
    self.max_length = max_length

    f = open("./drive/MyDrive/NLP_ass3/data/evidence.json", "r")
    self.evidences = json.load(f)
    f.close()

    self.evidences_ids = list(self.evidences.keys())

  def __len__(self):
    return len(self.evidences_ids)

  def __getitem__(self, index):
    evidences_id = self.evidences_ids[index]
    evidence = self.evidences[evidences_id]
    return [evidences_id, evidence]
    
  def collate_fn(self, batch):
    evidences_ids = []
    evidences = []

    for evidences_id, evidence in batch:
      evidences_ids.append(evidences_id)
      evidences.append(evidence.lower())

    evidences_text_token = self.tokenizer(
        evidences,
        max_length=self.max_length,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )

    encoding_dict = {"evidence_input_ids" : evidences_text_token["input_ids"],
               "evidence_attention_mask" : evidences_text_token["attention_mask"],
               "evidences_ids" : evidences_ids}

    return encoding_dict

### Auxiliary Functions

In [ ]:
def set_cuda(batch):
  for key in batch.keys():
    if key in ["query_input_ids", "evidence_input_ids", "query_attention_mask", "evidence_attention_mask"]:
      batch[key] = batch[key].cuda()

In [ ]:
# function to set the random seed
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)

### Crucial Functions

In [ ]:
# the function to get the evidence embeddings
def embed_evidence(evidence_DataLoader, model):
  # set the model to evaluate mode
  model.eval()
  e_embeddings = []
  e_ids = []
  for batch in tqdm(evidence_DataLoader):
    set_cuda(batch)
    # get the last hidden layer, detach the embedding from the logits
    e_outputs = model(input_ids=batch["evidence_input_ids"], attention_mask=batch["evidence_attention_mask"])
    e_logits = e_outputs.last_hidden_state
    e_embedding = e_logits[:, 0, :].detach()

    # transfer the e_embedding to cpu
    e_embedding_cpu = F.normalize(e_embedding).cpu()
    del e_logits, e_embedding

    # append the data to list
    e_embeddings.append(e_embedding_cpu)
    e_ids.extend(batch["evidences_ids"])

  e_embeddings = torch.cat(e_embeddings, dim=0).t()
  return e_embeddings, e_ids

In [ ]:
def evaluate(val_DataLoader, evidence_embeddings, evidence_ids, model, top_evidence):
  f_scores = []
  for batch in tqdm(val_DataLoader):
    set_cuda(batch)
    # get the last hidden layer, detach the embedding from the logits
    q_outputs = model(input_ids=batch["query_input_ids"], attention_mask=batch["query_attention_mask"])
    q_logits = q_outputs.last_hidden_state
    q_embedding = q_logits[:, 0, :]

    # transfer the q_embedding to cpu
    q_embedding_cpu = F.normalize(q_embedding).cpu()
    
    # get the evidences scores seperately and select the top ones
    similarity_scores = torch.mm(q_embedding_cpu, evidence_embeddings)
    batch_e_ids = torch.topk(similarity_scores, k=top_evidence, dim=1).indices.tolist()

    for index, data in enumerate(batch["datas"]):
        top_k_ids = batch_e_ids[index]
        top_e_ids = [evidence_ids[id] for id in top_k_ids]
        correct_evidence = [e_id for e_id in batch["evidences"][index] if e_id in top_e_ids]
        
        if len(correct_evidence) > 0:
            recall = len(correct_evidence) / len(batch["evidences"][index])
            precision = len(correct_evidence) / len(top_e_ids)
            f_score = (2 * precision * recall) / (precision + recall)
        else:
            f_score = 0
        f_scores.append(f_score)

  f_socre_final = np.mean(f_scores)
  print("\nEvidence Retrieval F-score = %.3f\n" % f_socre_final)

  # set the model back to train mode
  model.train()

  return f_socre_final

In [ ]:
def train(model_name, epoch, batch_size, max_length, evidence_samples, top_evidence, model_path):
  # initiate the wandb
  wandb.init(project="Task1 Evidence Retrieval", name="DPR_v2_roberta-base_DifferentLR")

  # set the random seed of the model
  setup_seed(42)

  # create the folder to save the model trained  
  month_date = datetime.now().strftime("%m-%d")
  checkpoints_dir_path = f"./drive/MyDrive/NLP_ass3/checkpoints_v2/{month_date}"
  if not os.path.exists(checkpoints_dir_path):
    os.makedirs(checkpoints_dir_path)
  
  # initialize the pretrained model
  query_model = AutoModel.from_pretrained(model_name)
  evidence_model = AutoModel.from_pretrained(model_name)

  if model_path != "":
      query_model.load_state_dict(torch.load(os.path.join("./drive/MyDrive/NLP_ass3/checkpoints_v2", model_path, "best_query.bin")))
      evidence_model.load_state_dict(torch.load(os.path.join("./drive/MyDrive/NLP_ass3/checkpoints_v2", model_path, "best_evidence.bin")))


  # use GPU to train the model
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  # print(device)
  query_model.to(device)
  evidence_model.to(device)
  # set model to train mode
  query_model.train()
  evidence_model.train()

  # get the tokenizer from the specific model
  tokenizer = AutoTokenizer.from_pretrained(model_name)

  # initialize the datasets and read to the DataLoader for feeding into the model
  train_dataset = TrainDataset("train", tokenizer, evidence_samples, max_length)
  val_dataset = ValidateDataset("dev", tokenizer, max_length)
  evidence_dataset = EvidenceDataset(tokenizer, max_length)

  train_DataLoader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8, collate_fn=train_dataset.collate_fn)
  val_DataLoader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=8, collate_fn=val_dataset.collate_fn)
  evidence_DataLoader = DataLoader(evidence_dataset, batch_size=128, shuffle=False, num_workers=8, collate_fn=evidence_dataset.collate_fn)

  # TODO: here to change the optimizer
  # TODO: adjust the lr
  query_optimizer = optim.Adam(query_model.parameters(), lr=2e-5)
  evidence_optimizer = optim.Adam(evidence_model.parameters(), lr=2e-5)
  query_scheduler = torch.optim.lr_scheduler.StepLR(query_optimizer, step_size=50, gamma=0.9, last_epoch=-1)
  evidence_scheduler = torch.optim.lr_scheduler.StepLR(evidence_optimizer, step_size=50, gamma=0.9, last_epoch=-1)


  # set some counter for training
  update_count = 0
  avg_loss = 0
  max_f_score = 0
  epoch_num = 0

  # get the evidence embedding for scoring
  evidence_embeddings, evidence_ids = embed_evidence(evidence_DataLoader, evidence_model)
  f_score = evaluate(val_DataLoader, evidence_embeddings, evidence_ids, query_model, top_evidence)
  wandb.log({"f_score": f_score}, step=update_count)

  # assign the f_score to max_f_score
  max_f_score = f_score
  
  for epoch in range(epoch):
    print("##################################################Training##################################################")
    # use to save the count of epoch update step
    for (idx, batch) in enumerate(tqdm(train_DataLoader)):
      # start training
      query_optimizer.zero_grad()
      evidence_optimizer.zero_grad()
      
      # put the data in batch into cuda
      set_cuda(batch)

      # get the embeddings of the q and e according 
      q_outputs = query_model(input_ids=batch["query_input_ids"], attention_mask=batch["query_attention_mask"])
      q_logits = q_outputs.last_hidden_state

      e_outputs = evidence_model(input_ids=batch["evidence_input_ids"], attention_mask=batch["evidence_attention_mask"])
      e_logits = e_outputs.last_hidden_state
      
      q_embeddings = q_logits[:, 0, :]
      e_embeddings = e_logits[:, 0, :]

      # normalize the embeddings
      q_embeddings = F.normalize(q_embeddings)
      e_embeddings = F.normalize(e_embeddings)

      # calculate the cosine similarity between the queries and evidences
      cos_similarities = torch.mm(q_embeddings, e_embeddings.t())
      # prevent overflow, accelarate back propogation
      log_soft_scores = - F.log_softmax(cos_similarities * 50, dim=1)

      loss_list = []
      start_index = 0
      for index, answer_len in enumerate(batch["answer_lens"]):
        end_index = start_index + answer_len
        current_loss = torch.mean(log_soft_scores[index, start_index:end_index])
        loss_list.append(current_loss)
        start_index = end_index

      loss = torch.stack(loss_list).mean()
      
      # backward the loss, update the parameters in the model
      loss.backward()
      avg_loss = avg_loss + loss.item()

      # mitigate the problem of exploding gradients
      nn.utils.clip_grad_norm_(query_model.parameters(), 1)
      nn.utils.clip_grad_norm_(evidence_model.parameters(), 1)
      
      # optimize the model and lr
      query_optimizer.step()
      evidence_optimizer.step()
      query_scheduler.step()
      evidence_scheduler.step()


      update_count += 1

      wandb_freq = 20
      if update_count % wandb_freq == 0:
        wandb.log({"loss": avg_loss / wandb_freq}, step=update_count)
        avg_loss = 0

      del loss, cos_similarities, q_embeddings, e_embeddings

    print("##################################################Evaluate##################################################")
    # finish one epoch, add one to the epoch_num
    epoch_num += 1
    # evaluate the model every epoch and save the best one that have the best f_score
    evidence_embeddings, evidence_ids = embed_evidence(evidence_DataLoader, evidence_model)
    f_score = evaluate(val_DataLoader, evidence_embeddings, evidence_ids, query_model, top_evidence)
    wandb.log({"f_score": f_score}, step=update_count)

    if f_score > max_f_score:
      max_f_score = f_score
      torch.save(query_model.state_dict(), os.path.join(checkpoints_dir_path, "best_query.bin"))
      torch.save(evidence_model.state_dict(), os.path.join(checkpoints_dir_path, "best_evidence.bin"))
      print("\nThis is the", epoch_num, "epoch", "the max f_score is", max_f_score)
  # finish the wandb
  wandb.finish()

In [ ]:
def predict(model_name, batch_size, max_length, top_evidence, model_path):
  # get the tokenizer from the specific model
  tokenizer = AutoTokenizer.from_pretrained(model_name)

  # initialize the pretrained model
  query_model = AutoModel.from_pretrained(model_name)
  evidence_model = AutoModel.from_pretrained(model_name)

  # load the best finetuned parameters
  assert model_path
  # print(os.path.join("./drive/MyDrive/NLP_ass3/checkpoints_v2", model_path, "best_state_dict.bin"))
  query_model.load_state_dict(torch.load(os.path.join("./drive/MyDrive/NLP_ass3/checkpoints_v2", model_path, "best_query.bin")))
  evidence_model.load_state_dict(torch.load(os.path.join("./drive/MyDrive/NLP_ass3/checkpoints_v2", model_path, "best_evidence.bin")))

  # use GPU, and set the model to evaluate mode
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  # print(device)
  query_model.to(device)
  evidence_model.to(device)

  query_model.eval()
  evidence_model.eval()

  # load the test and evidence datasets
  test_dataset = ValidateDataset("test", tokenizer, max_length)
  evidence_dataset = EvidenceDataset(tokenizer, max_length)

  test_DataLoader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=8, collate_fn=test_dataset.collate_fn)
  evidence_DataLoader = DataLoader(evidence_dataset, batch_size=128, shuffle=False, num_workers=8, collate_fn=evidence_dataset.collate_fn)


  # utilize embed_evidence function to get the evidences embeddings
  evidence_embeddings, evidence_ids = embed_evidence(evidence_DataLoader, evidence_model)

  output = {}
  for batch in tqdm(test_DataLoader):
    set_cuda(batch)
    # get the last hidden layer, detach the embedding from the logits
    q_outputs = query_model(input_ids=batch["query_input_ids"], attention_mask=batch["query_attention_mask"])
    q_logits = q_outputs.last_hidden_state
    q_embedding = q_logits[:, 0, :]

    # transfer the q_embedding to cpu
    q_embedding_cpu = F.normalize(q_embedding).cpu()
    
    # get the evidences scores seperately and select the top ones
    similarity_scores = torch.mm(q_embedding_cpu, evidence_embeddings)
    batch_e_ids = torch.topk(similarity_scores, k=top_evidence, dim=1).indices.tolist()

    for index, data in enumerate(batch["datas"]):
      top_k_ids = batch_e_ids[index]
      data["evidences"] = [evidence_ids[id] for id in top_k_ids]
      claim_id = batch["claim_ids"][index]
      output[claim_id] = data
  fout = open("./drive/MyDrive/NLP_ass3/predict/v2/test-claims-retrieved.json", 'w')
  json.dump(output, fout)
  fout.close()

In [ ]:
# training settings
epoch = 10
batch_size = 8
max_length = 128
# max_length = 512
evidence_samples = 64
model_path = ""
# TODO, adjust the top_evidence number
top_evidence = 3
# initialize the pretrained model
# TODO, use different pretrained model
# model_name = "bert-base-uncased"
model_name = "roberta-base"

train(model_name, epoch, batch_size, max_length, evidence_samples, top_evidence, model_path)

# predict(model_name, batch_size, max_length, top_evidence, model_path)



<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaMod

100%|██████████| 20/20 [00:07<00:00,  2.75it/s]



Evidence Retrieval F-score = 0.024

##################################################Training##################################################


...

100%|██████████| 154/154 [00:41<00:00,  3.73it/s]


##################################################Evaluate##################################################


100%|██████████| 20/20 [00:07<00:00,  2.75it/s]


Evidence Retrieval F-score = 0.120



f_score,▁▃▅▆▆▆█▇▇▇▆
loss,█▅▅▅▄▃▃▃▂▃▂▂▂▂▂▂▁▂▂▂▂▂▂▁▁▁▁▁▁▂▁▁▁▁▂▁▁▁▁▁
f_score,0.1199
loss,1.24609
